In [ ]:
import os
import zipfile
import shutil
from subprocess import getoutput
from IPython.utils import capture
import random
import concurrent.futures
from tqdm import tqdm
from PIL import Image
import re
import json
import glob
import gdown
import requests
import subprocess
from urllib.parse import urlparse, unquote
from pathlib import Path
import toml

!pip install -q torch==2.0.0+cu118 torchvision==0.15.1+cu118 torchaudio==2.0.1+cu118 torchtext==0.15.1 torchdata==0.6.0 --extra-index-url https://download.pytorch.org/whl/cu118 -U
!pip install -q xformers==0.0.19 triton==2.0.0 -U

#root_dir
root_dir = "/content"
deps_dir = os.path.join(root_dir,"deps")
repo_dir = os.path.join(root_dir,"Kohya-Colab")
training_dir = os.path.join(root_dir,"Dreamboot-Config")
pretrained_model = os.path.join(root_dir,"pretrained_model")
vae_dir = os.path.join(root_dir,"vae")
config_dir = os.path.join(training_dir,"config")

#repo_dir
accelerate_config = os.path.join(repo_dir, "accelerate_config/config.yaml")
tools_dir = os.path.join(repo_dir,"tools")
finetune_dir = os.path.join(repo_dir,"finetune")

for store in ["root_dir", "deps_dir", "repo_dir", "training_dir", "pretrained_model", "vae_dir", "accelerate_config", "tools_dir", "finetune_dir", "config_dir"]:
  with capture.capture_output() as cap:
    %store {store}
    del cap

repo_url = "https://github.com/phamhungd/Kohya-Colab"
branch = ""

for dir in [deps_dir, training_dir, config_dir, pretrained_model, vae_dir]:
  os.makedirs(dir, exist_ok=True)

def clone_repo(url):
  if not os.path.exists(repo_dir):
    os.chdir(root_dir)
    !git clone {url} {repo_dir}
  else:
    os.chdir(repo_dir)
    !git pull origin {branch} if branch else !git pull

clone_repo(repo_url)

if branch:
  os.chdir(repo_dir)
  status = os.system(f"git checkout {branch}")
  if status != 0:
    raise Exception("Failed to checkout branch or commit")

os.chdir(repo_dir)

def ubuntu_deps(url, name, dst):
  with capture.capture_output() as cap:
    !wget -q --show-progress {url}
    with zipfile.ZipFile(name, 'r') as deps:
      deps.extractall(dst)
    !dpkg -i {dst}/*
    os.remove(name)
    shutil.rmtree(dst)
    del cap

def install_dependencies():
   !pip -q install --upgrade -r requirements.txt

   !pip install xformers triton

   from accelerate.utils import write_basic_config
   if not os.path.exists(accelerate_config):
      write_basic_config(save_location=accelerate_config)

os.chdir(repo_dir)
ubuntu_deps("https://huggingface.co/Linaqruf/fast-repo/resolve/main/deb-libs.zip", "deb-libs.zip", deps_dir)
install_dependencies()

!apt --fix-broken install


print(f"Your train data directory : {train_data_dir}")

os.chdir(finetune_dir)

config = {
   "_train_data_dir": train_data_dir,
   "batch_size": 8,
   "repo_id": "SmilingWolf/wd-v1-4-convnextv2-tagger-v2",
   "recursive": True,
   "remove_underscore": True,
   "general_threshold": Threshold,
   "character_threshold": 0.35,
   "caption_extension": ".txt",
   "max_data_loader_n_workers": 2,
   "debug": True,
}

args = ""
for k, v in config.items():
   if k.startswith("_"):
      args += f'"{v}" '
   elif isinstance(v, str):
      args += f'--{k}="{v}" '
   elif isinstance(v, bool) and v:
      args += f"--{k} "
   elif isinstance(v, float) and not isinstance(v, bool):
      args += f"--{k}={v} "
   elif isinstance(v, int) and not isinstance(v, bool):
      args += f"--{k}={v} "

final_args = f"python tag_images_by_wd14_tagger.py {args}"
if not NoAutoCaption :
    !{final_args}

os.chdir(root_dir)

extension = ".txt"
custom_tag = CustomCaption

def read_file(filename):
   with open(filename, "r") as f:
         contents = f.read()
   return contents

def write_file(filename, contents):
   with open(filename, "w") as f:
         f.write(contents)

def process_tags(filename, custom_tag, append, remove_tag):
   contents = read_file(filename)
   tags = [tag.strip() for tag in contents.split(',')]
   custom_tags = [tag.strip() for tag in custom_tag.split(',')]

   for custom_tag in custom_tags:
         custom_tag = custom_tag.replace("_", " ")
         if remove_tag:
            while custom_tag in tags:
               tags.remove(custom_tag)
         else:
            if custom_tag not in tags:
               if append:
                     tags.append(custom_tag)
               else:
                     tags.insert(0, custom_tag)

   contents = ', '.join(tags)
   write_file(filename, contents)

def process_directory(train_data_dir, tag, append, remove_tag, recursive):
   for filename in os.listdir(train_data_dir):
         file_path = os.path.join(train_data_dir, filename)
         if os.path.isdir(file_path) and recursive:
            process_directory(file_path, tag, append, remove_tag, recursive)
         elif filename.endswith(extension):
            process_tags(file_path, tag, append, remove_tag)

if not any(
   [filename.endswith(extension) for filename in os.listdir(train_data_dir)]
):
   for filename in os.listdir(train_data_dir):
         if filename.endswith((".png", ".jpg", ".jpeg", ".webp", ".bmp")):
            open(
               os.path.join(train_data_dir, filename.split(".")[0] + extension),
               "w",
            ).close()
if not NoAutoCaption :
    process_directory(train_data_dir, custom_tag, False, False, True)

#3.Setting

MODEL_URLS = {
    "RealisticVision-v51" : "https://huggingface.co/phamhungd/GuoZovya/resolve/main/RealisticVision-v51.safetensors",
    "Anything-v3" : "https://huggingface.co/cag/anything-v3-1/resolve/main/anything-v3-1.safetensors",
    "Chilloutmix" : "https://huggingface.co/Linaqruf/stolen/resolve/main/pruned-models/chillout_mix-pruned.safetensors",
}
MODEL_URL = MODEL_URLS.get(Model, Model)
drive_dir = os.path.join(root_dir, "drive/MyDrive")
def get_supported_extensions():
    return tuple([".ckpt", ".safetensors", ".pt", ".pth"])

def get_filename(url, quiet=True):
    extensions = get_supported_extensions()

    if url.startswith(drive_dir) or url.endswith(tuple(extensions)):
        filename = os.path.basename(url)
    else:
        response = requests.get(url, stream=True)
        response.raise_for_status()

        if 'content-disposition' in response.headers:
            content_disposition = response.headers['content-disposition']
            filename = re.findall('filename="?([^"]+)"?', content_disposition)[0]
        else:
            url_path = urlparse(url).path
            filename = unquote(os.path.basename(url_path))

    if filename.endswith(tuple(get_supported_extensions())):
        return filename
    else:
        return None

def get_most_recent_file(directory):
    files = glob.glob(os.path.join(directory, "*"))
    if not files:
        return None
    most_recent_file = max(files, key=os.path.getmtime)
    basename = os.path.basename(most_recent_file)

    return most_recent_file

def parse_args(config):
    args = []

    for k, v in config.items():
        if k.startswith("_"):
            args.append(f"{v}")
        elif isinstance(v, str) and v is not None:
            args.append(f'--{k}={v}')
        elif isinstance(v, bool) and v:
            args.append(f"--{k}")
        elif isinstance(v, float) and not isinstance(v, bool):
            args.append(f"--{k}={v}")
        elif isinstance(v, int) and not isinstance(v, bool):
            args.append(f"--{k}={v}")

    return args
def aria2_download(dir, filename, url):
    aria2_config = {
        "console-log-level"         : "error",
        "summary-interval"          : 10,
        "continue"                  : True,
        "max-connection-per-server" : 16,
        "min-split-size"            : "1M",
        "split"                     : 16,
        "dir"                       : dir,
        "out"                       : filename,
        "_url"                      : url,
    }
    aria2_args = parse_args(aria2_config)
    subprocess.run(["aria2c", *aria2_args])

def gdown_download(url, dst, filepath):
    if "/uc?id/" in url:
        return gdown.download(url, filepath, quiet=False)
    elif "/file/d/" in url:
        return gdown.download(url=url, output=filepath, quiet=False, fuzzy=True)
    elif "/drive/folders/" in url:
        os.chdir(dst)
        return gdown.download_folder(url, quiet=True, use_cookies=False)

def download(url, dst):
    print(f"Starting downloading from {url}")
    filename = get_filename(url)
    filepath = os.path.join(dst, filename)

    if "drive.google.com" in url:
        gdown = gdown_download(url, dst, filepath)
    else:
        if "huggingface.co" in url and "/blob/" in url:
            url = url.replace("/blob/", "/resolve/")
        aria2_download(dst, filename, url)

    print(f"Download finished: {filepath}")
    return filepath

def get_gpu_name():
    try:
        return subprocess.check_output("nvidia-smi --query-gpu=name --format=csv,noheader,nounits", shell=True).decode('ascii').strip()
    except:
        return None

def main():
    global model_path, vae_path
    model_path, vae_path = None, None
    download_targets = {
        "model": (MODEL_URL, pretrained_model),
    }
    for target, (url, dst) in download_targets.items():
        if url and not url.startswith(f"PASTE {target.upper()} URL OR GDRIVE PATH HERE"):
            filepath = download(url, dst)
            if target == "model":
                model_path = filepath
            print()
    if model_path:
        print(f"Selected model: {model_path}")

if Model.startswith("/content/drive/"):
  model_path = Model
  print(f"Diffusers model is loaded : {Model}")
else:
  main()

!aria2c --console-log-level=error --summary-interval=10 -c -x 16 -k 1M -s 16 https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt -d /content/VAE -o VAE84EMA.vae.pt
vae = "/content/VAE/VAE84EMA.vae.pt"

#Dataset Config

activation_word = "sd vn"
caption_extension = ".txt"
token_to_captions = False
dataset_repeats = Repeats
keep_tokens = 0
flip_aug = False

if ',' in activation_word or ' ' in activation_word:
    words = activation_word.replace(',', ' ').split()
    class_token = words[-1]


def read_file(filename):
    with open(filename, "r") as f:
        contents = f.read()
    return contents


def write_file(filename, contents):
    with open(filename, "w") as f:
        f.write(contents)


def get_supported_images(folder):
    supported_extensions = (".png", ".jpg", ".jpeg", ".webp", ".bmp")
    return [file for ext in supported_extensions for file in glob.glob(f"{folder}/*{ext}")]


def get_subfolders_with_supported_images(folder):
    subfolders = [os.path.join(folder, subfolder) for subfolder in os.listdir(folder) if os.path.isdir(os.path.join(folder, subfolder))]
    return [subfolder for subfolder in subfolders if len(get_supported_images(subfolder)) > 0]


def process_tags(filename, custom_tag, remove_tag):
    contents = read_file(filename)
    tags = [tag.strip() for tag in contents.split(',')]
    custom_tags = [tag.strip() for tag in custom_tag.split(',')]

    for custom_tag in custom_tags:
        custom_tag = custom_tag.replace("_", " ")
        # if remove_tag:
        #     while custom_tag in tags:
        #         tags.remove(custom_tag)
        # else:
        if custom_tag not in tags:
            tags.insert(0, custom_tag)

    contents = ', '.join(tags)
    write_file(filename, contents)


def process_folder_recursively(folder):
    for root, _, files in os.walk(folder):
        for file in files:
            if file.endswith(caption_extension):
                file_path = os.path.join(root, file)
                extracted_class_token = get_class_token_from_folder_name(root, folder)
                train_supported_images = get_supported_images(train_data_dir)
                tag = extracted_class_token if extracted_class_token else activation_word if train_supported_images else ""
                if not tag == "":
                    process_tags(file_path, tag, remove_tag=(not token_to_captions))


def get_num_repeats(folder):
    folder_name = os.path.basename(folder)
    try:
        repeats, _ = folder_name.split('_', 1)
        num_repeats = int(repeats)
    except ValueError:
        num_repeats = dataset_repeats

    return num_repeats


def get_class_token_from_folder_name(folder, parent_folder):
    if folder == parent_folder:
        return class_token

    folder_name = os.path.basename(folder)
    try:
        _, concept = folder_name.split('_', 1)
        return concept
    except ValueError:
        return ""

train_supported_images = get_supported_images(train_data_dir)
train_subfolders = get_subfolders_with_supported_images(train_data_dir)

subsets = []
config = {
    "general": {
        "enable_bucket": True,
        "caption_extension": caption_extension,
        "shuffle_caption": True,
        "keep_tokens": keep_tokens,
        "bucket_reso_steps": 64,
        "bucket_no_upscale": False,
    },
    "datasets": [
        {
            "resolution": resolution,
            "min_bucket_reso": 320 if resolution > 640 else 256,
            "max_bucket_reso": 1280 if resolution > 640 else 1024,
            "caption_dropout_rate": 0,
            "caption_tag_dropout_rate": 0,
            "caption_dropout_every_n_epochs": 0,
            "flip_aug": flip_aug,
            "color_aug": False,
            "face_crop_aug_range": None,
            "subsets": subsets,
        }
    ],
}

if token_to_captions and keep_tokens < 2:
    keep_tokens = 1

process_folder_recursively(train_data_dir)

if train_supported_images:
    subsets.append({
        "image_dir": train_data_dir,
        "class_tokens": activation_word,
        "num_repeats": dataset_repeats,
    })

for subfolder in train_subfolders:
    num_repeats = get_num_repeats(subfolder)
    extracted_class_token = get_class_token_from_folder_name(subfolder, train_data_dir)
    subsets.append({
        "image_dir": subfolder,
        "class_tokens": extracted_class_token if extracted_class_token else None,
        "num_repeats": num_repeats,
    })

for subset in subsets:
    if not glob.glob(f"{subset['image_dir']}/*.txt"):
        subset["class_tokens"] = activation_word

dataset_config = os.path.join(config_dir, "dataset_config.toml")

for key in config:
    if isinstance(config[key], dict):
        for sub_key in config[key]:
            if config[key][sub_key] == "":
                config[key][sub_key] = None
    elif config[key] == "":
        config[key] = None

config_str = toml.dumps(config)

with open(dataset_config, "w") as f:
    f.write(config_str)

print(config_str)

#Config

conv_dim = 4
conv_alpha = 1

network_module = "networks.lora"
network_args = ""

config = {
   "model_arguments": {
         "v2": False,
         "v_parameterization": False,
         "pretrained_model_name_or_path": model_path,
         "vae": vae,
   },
   "additional_network_arguments": {
         "no_metadata": False,
         "unet_lr": float(unet_lr),
         "text_encoder_lr": float(text_encoder_lr),
         "network_module": network_module,
         "network_dim": 64,
         "network_alpha": 32,
         "training_comment": "sdvn.me",
   },
   "optimizer_arguments": {
         "optimizer_type": "AdamW8bit",
         "learning_rate": unet_lr,
         "max_grad_norm": 1.0,
         "lr_scheduler": "constant",
   },
   "dataset_arguments": {
         "cache_latents": True,
         "debug_dataset": False,
         "vae_batch_size": Batch_size,
   },
   "training_arguments": {
         "output_dir": output_dir,
         "output_name": Loraname,
         "save_precision": "fp16",
         "save_every_n_epochs": save_n_epochs_type_value,
         "train_batch_size": Batch_size,
         "max_token_length": 225,
         "mem_eff_attn": False,
         "xformers": True,
         "max_train_epochs": num_epochs,
         "max_data_loader_n_workers": 8,
         "persistent_data_loader_workers": True,
         "gradient_checkpointing": False,
         "gradient_accumulation_steps": 1,
         "mixed_precision": "fp16",
         "clip_skip": 2,
         "logging_dir": "/content/Dreamboot-Config/logs",
         "log_prefix": Loraname,
         "lowram": True,
         "training_comment" : "train by sdvn.me train 1click",
   },
   "sample_prompt_arguments": {
         "sample_every_n_steps": 200,
         "sample_every_n_epochs": 1,
         "sample_sampler": "euler",
   },
   "dreambooth_arguments": {
         "prior_loss_weight": 1,
   },
   "saving_arguments": {
         "save_model_as": "safetensors",
   },
}
SamplePrompt = f"{Loraname},front view, masterpiece,best quality"
sample_str = f"""
  {SamplePrompt}\
  --n lowres, bad anatomy, bad hands, text, error, missing fingers, extra digit, fewer digits, cropped, worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry \
  --w 512 \
  --h 768 \
  --l 7 \
  --s 28
"""
config_path = os.path.join(config_dir, "config_file.toml")
prompt_path = os.path.join(config_dir, "sample_prompt.txt")

for key in config:
    if isinstance(config[key], dict):
        for sub_key in config[key]:
            if config[key][sub_key] == "":
                config[key][sub_key] = None
    elif config[key] == "":
        config[key] = None

config_str = toml.dumps(config)

def write_file(filename, contents):
    with open(filename, "w") as f:
        f.write(contents)

write_file(config_path, config_str)
write_file(prompt_path, sample_str)

print(config_str)

os.chdir(repo_dir)


train_file = "train_network.py"
ConfigFolder = "/content/Dreamboot-Config/config"
sample_prompt = f"{ConfigFolder}/sample_prompt.txt"
config_file = f"{ConfigFolder}/config_file.toml"
dataset_config = f"{ConfigFolder}/dataset_config.toml"
accelerate_conf = {
    "config_file" : accelerate_config,
    "num_cpu_threads_per_process" : 1,
}

train_conf = {
    "sample_prompts" : sample_prompt,
    "dataset_config" : dataset_config,
    "config_file" : config_file
}

def train(config):
    args = ""
    for k, v in config.items():
        if k.startswith("_"):
            args += f'"{v}" '
        elif isinstance(v, str):
            args += f'--{k}="{v}" '
        elif isinstance(v, bool) and v:
            args += f"--{k} "
        elif isinstance(v, float) and not isinstance(v, bool):
            args += f"--{k}={v} "
        elif isinstance(v, int) and not isinstance(v, bool):
            args += f"--{k}={v} "

    return args

accelerate_args = train(accelerate_conf)
train_args = train(train_conf)
final_args = f"accelerate launch {accelerate_args} {train_file} {train_args}"
